In [1]:
from data_processor_lib import Processor

data_src = "https://media.geeksforgeeks.org/wp-content/uploads/nba.csv"

In [2]:
processor = Processor()
processor.read_csv(data_src)

Profiler initialized


In [3]:
processor.columns()

Index(['Name', 'Team', 'Number', 'Position', 'Age', 'Height', 'Weight',
       'College', 'Salary'],
      dtype='object')

In [4]:
processor.head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0


In [5]:
processor.overview()

,Value
Total Columns,9.00
Total Rows,458.00
Incomplete Rows Count,94.00
Incomplete Rows (%),20.52
Incomplete Columns Count,9.00
Total Data Count,4018.00
Missing Data Count,104.00
Missing Data (%),2.59


In [6]:
processor.insights()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
Valid Data Count,457.0,457.0,457.0,457.0,457.0,457.0,457.0,373.0,446.0
Missing Data Count,1.0,1.0,1.0,1.0,1.0,1.0,1.0,85.0,12.0
Missing Data (%),0.22,0.22,0.22,0.22,0.22,0.22,0.22,22.79,2.69
Data Type,object,object,float64,object,float64,object,float64,object,float64
Mean,-,-,17.68,-,26.94,-,221.52,-,4842684.11
Median,-,-,13.0,-,26.0,-,220.0,-,2839073.0
Min,-,-,0.0,-,19.0,-,161.0,-,30888.0
Max,-,-,99.0,-,40.0,-,307.0,-,25000000.0
Standard Deviation,-,-,15.97,-,4.4,-,26.37,-,5229237.6
25% Quantile,-,-,5.0,-,24.0,-,200.0,-,1044792.25
